<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_Sec49AdvanceDL/blob/master/Assignment_4/Assignment_4_extra_a)_zeroshot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extra points (optional) - will go to extra credit :

a) Write colab to demo the use of pre trained multi model multi task learning model from ibm (refer to my slides - omninet). Demo zeroshot learning there.

In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
dataset = 'CUB'

In [ ]:
res101 = scipy.io.loadmat('/content/gdrive/dataset/xlsa17/data/'+dataset+'/res101.mat')
att_splits = scipy.io.loadmat('/content/gdrive/dataset/xlsa17/data/'+dataset+'/att_splits.mat')

In [ ]:
res101.keys()

In [ ]:
att_splits.keys()

In [ ]:
trainval_loc = 'trainval_loc'
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

In [ ]:
labels = res101['labels']
labels_train = labels[np.squeeze(att_splits[train_loc]-1)]
labels_val = labels[np.squeeze(att_splits[val_loc]-1)]
labels_trainval = labels[np.squeeze(att_splits[trainval_loc]-1)]
labels_test = labels[np.squeeze(att_splits[test_loc]-1)]

In [ ]:
labels_train[:10,:]

In [ ]:
unique_labels = np.unique(labels)
unique_labels

In [ ]:
train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
trainval_labels_seen = np.unique(labels_trainval)
test_labels_unseen = np.unique(labels_test)

In [ ]:
print("Number of overlapping classes between train and val:",len(set(train_labels_seen).intersection(set(val_labels_unseen))))
print("Number of overlapping classes between trainval and test:",len(set(trainval_labels_seen).intersection(set(test_labels_unseen))))

In [ ]:
i = 0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i    
    i = i+1
j = 0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j = j+1
k = 0
for labels in trainval_labels_seen:
    labels_trainval[labels_trainval == labels] = k
    k = k+1
l = 0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = l
    l = l+1

Let us denote the features X ∈ [d×m] available at training stage, where d is the dimensionality
of the data, and m is the number of instances. We are useing resnet features which are extracted from `CUB` dataset.

In [ ]:
X_features = res101['features']
train_vec = X_features[:,np.squeeze(att_splits[train_loc]-1)]
val_vec = X_features[:,np.squeeze(att_splits[val_loc]-1)]
trainval_vec = X_features[:,np.squeeze(att_splits[trainval_loc]-1)]
test_vec = X_features[:,np.squeeze(att_splits[test_loc]-1)]

In [ ]:
print("Features for train:", train_vec.shape)
print("Features for val:", val_vec.shape)
print("Features for trainval:", trainval_vec.shape)
print("Features for test:", test_vec.shape)

#### Normalize the vectors

In [ ]:
def normalization(vec,mean,std):
    sol = vec - mean
    sol1 = sol/std
    return sol1

In [ ]:
#Signature matrix
signature = att_splits['att']
train_sig = signature[:,(train_labels_seen)-1]
val_sig = signature[:,(val_labels_unseen)-1]
trainval_sig = signature[:,(trainval_labels_seen)-1]
test_sig = signature[:,(test_labels_unseen)-1]



```
 Horse      Zebra
[0.00354613 0.        ] Domestic_animal
[0.13829921 0.20209503] 4_legged
[0.06560347 0.04155225] carnivore
```

In [ ]:
print(train_sig[3:6,:2])

In [ ]:
print("Signature for train:", train_sig.shape)
print("Signature for val:", val_sig.shape)
print("Signature for trainval:", trainval_sig.shape)
print("Signature for test:", test_sig.shape)

In [ ]:
#params for train and val set
m_train = labels_train.shape[0]
n_val = labels_val.shape[0]
z_train = len(train_labels_seen)
z1_val = len(val_labels_unseen)

#params for trainval and test set
m_trainval = labels_trainval.shape[0]
n_test = labels_test.shape[0]
z_trainval = len(trainval_labels_seen)
z1_test = len(test_labels_unseen)

The ground truth is a one-hot encoded vector

In [ ]:
#ground truth for train and val set
gt_train = 0*np.ones((m_train, z_train))
gt_train[np.arange(m_train), np.squeeze(labels_train)] = 1

#grountruth for trainval and test set
gt_trainval = 0*np.ones((m_trainval, z_trainval))
gt_trainval[np.arange(m_trainval), np.squeeze(labels_trainval)] = 1

In [ ]:
gt_train[:1,:100]

In [ ]:
#train set
d_train = train_vec.shape[0]
a_train = train_sig.shape[0]

#Weights
V = np.zeros((d_train,a_train))

In [ ]:
#trainval set
d_trainval = trainval_vec.shape[0]
a_trainval = trainval_sig.shape[0]
W = np.zeros((d_trainval,a_trainval))

#Note: These hyper-parameters were found using the code snippet available below
gamm1 = 3
alph1 = 0

The one-line code solution proposed.
```
V = inverse(XX' + γI) XYS' inverse(SS' + λI)
```



In [ ]:
part_1_test = np.linalg.pinv(np.matmul(trainval_vec, trainval_vec.transpose()) + (10**alph1)*np.eye(d_trainval))
part_0_test = np.matmul(np.matmul(trainval_vec,gt_trainval),trainval_sig.transpose())
part_2_test = np.linalg.pinv(np.matmul(trainval_sig, trainval_sig.transpose()) + (10**gamm1)*np.eye(a_trainval))

W = np.matmul(np.matmul(part_1_test,part_0_test),part_2_test)

For inference stage, 
```
argmax(x'VS)
```
Where S is the signature matrix of the test_set

In [ ]:
#predictions
outputs_1 = np.matmul(np.matmul(test_vec.transpose(),W),test_sig)
preds_1 = np.array([np.argmax(output) for output in outputs_1])

In [ ]:
cm = confusion_matrix(labels_test, preds_1)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
avg = sum(cm.diagonal())/len(test_labels_unseen)
print("The top 1% accuracy is:", avg*100)

In [ ]:
accu = 0.10
alph1 = 4
gamm1 = 1
for alpha in range(-3, 4):
    for gamma in range(-3,4):
        #One line solution
        part_1 = np.linalg.pinv(np.matmul(train_vec, train_vec.transpose()) + (10**alpha)*np.eye(d_train))
        part_0 = np.matmul(np.matmul(train_vec,gt_train),train_sig.transpose())
        part_2 = np.linalg.pinv(np.matmul(train_sig, train_sig.transpose()) + (10**gamma)*np.eye(a_train))

        V = np.matmul(np.matmul(part_1,part_0),part_2)
        #print(V)

        #predictions
        outputs = np.matmul(np.matmul(val_vec.transpose(),V),val_sig)
        preds = np.array([np.argmax(output) for output in outputs])

        #print(accuracy_score(labels_val,preds))
        cm = confusion_matrix(labels_val, preds)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        avg = sum(cm.diagonal())/len(val_labels_unseen)
        #print("Avg:", avg, alpha, gamma)

        if avg > accu:
            accu = avg
            alph1 = alpha
            gamm1 = gamma
print(alph1, gamm1)